In [1]:
def get_dtype_lists(data,features):
    output = {}
    for f in features:
        dtype = str(data[f].dtype)
        if dtype not in output.keys(): 
            output[dtype] = [f]
        else: 
            output[dtype] += [f]
    return output

In [2]:
def get_count(data, tuples):
    data = data.copy()
    data['Count'] = data.index
    ordered_categories = [category for (category, value) in tuples]
    counts = data.groupby(ordered_categories).count()['Count']
    count = counts
    for tup in tuples: 
        count = count[tup[1]]
    return count

In [3]:
def get_support(data, tuples):
    support = get_count(data,tuples)
    return support/len(data)

In [4]:
def get_confidence(data, predictors, outcomes):
    numerator = get_support(data,predictors+outcomes)
    denominator = get_support(data, predictors)
    return numerator / denominator

In [5]:
def get_lift(data,tuples):
    numerator = get_support(data,tuples)
    denominator = 1 
    for tup in tuples: 
        denominator *= get_support(data,[tup])
    return numerator /denominator

In [6]:
def get_leverage(data, tuples):
    support = get_support(data, tuples)
    support_p = get_support(data, [tuples[0]])
    support_q = get_support(data, [tuples[1]])
    return support - support_p*support_q

In [7]:
def get_conviction(data, predictors, outcomes):
    numerator = 1 - get_support(data, outcomes)
    denominator = 1 - get_confidence(data, predictors, outcomes)
    if denominator == 0:
        return float('inf')
    return numerator / denominator

In [8]:
def get_category_value_tuples(data, categories):
    tuples = []
    for category in categories:
        for value in data[category].unique():
            tuples += [(category,value)]
    return tuples

In [9]:
def jaccard_distance(g, p1, p2):
    neighbors1 = set()
    neighbors2 = set()
    
    # Create a URIRef for the property
    if p1 in clean_object_properties:
        p1 = ont_namespace['has' + p1]
    elif p1 in clean_data_properties:
        p1 = ont_namespace[p1]
    else:
        return None
    
    # Create a URIRef for the property
    if p2 in clean_object_properties:
        p2 = ont_namespace['has' + p2]
    elif p2 in clean_data_properties:
        p2 = ont_namespace[p2]
    else:
        return None
    
    #iterate through the graph and add neighbors
    for s, p, o in g.triples((None, None, None)):
        if p == p1:
            neighbors1.add(o)
            neighbors1.add(s)
        elif p == p2:
            neighbors2.add(o)
            neighbors2.add(s)

    intersection = neighbors1.intersection(neighbors2)
    union = neighbors1.union(neighbors2)
    return 1.0 - len(intersection) / len(union)

In [10]:
import pandas as pd 
from tqdm import tqdm
import rdflib
from rdflib import Graph, Namespace, URIRef, OWL, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD
from rdflib import RDF, RDFS, OWL, Namespace, Graph

In [11]:
# create a new RDF graph
graph = Graph()

# add your ontology to the graph
graph.parse('C:/Users/mouad/Desktop/upwork/US Birth/data/BirthOntology.owl', format='xml')

<Graph identifier=N43d302854ba947649223091d5e72276a (<class 'rdflib.graph.Graph'>)>

In [12]:
# Print all the namespaces defined in the graph
for prefix, uri in graph.namespace_manager.namespaces():
    print(f"{prefix}: {uri}")

owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-schema#
xsd: http://www.w3.org/2001/XMLSchema#
xml: http://www.w3.org/XML/1998/namespace
: http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#
orw: http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#hasCleftLipw/orw/
swrl: http://www.w3.org/2003/11/swrl#
swrla: http://swrl.stanford.edu/ontologies/3.3/swrla.owl#
swrlb: http://www.w3.org/2003/11/swrlb#


In [13]:
# create a namespace for your ontology
ont_namespace = Namespace('http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#')

In [14]:
# Get all object properties
object_properties = list(graph.subjects(RDF.type, OWL.ObjectProperty))
clean_object_properties =  [str(object_propertiy).split('has')[1] for object_propertiy in object_properties]
# Get all data properties
data_properties = list(graph.subjects(RDF.type, OWL.DatatypeProperty))
clean_data_properties = [str(data_property).split('#')[1] for data_property in data_properties]

In [15]:
# Define the chunk size
chunk_size = 100000

# specify the file path
file_path = 'US_birth.csv'

# Initialize an empty list to store chunks
chunks = []

# read the file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate the chunks to a single dataframe
df = pd.concat(chunks, ignore_index=True)
df= df.drop('Unnamed: 0',axis= 1)
df.head()

C:\Users\mouad\AppData\Local\Temp\ipykernel_3292\3464066820.py:11: DtypeWarning: Columns (59,68,92,93,94,99) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_3292\3464066820.py:11: DtypeWarning: Columns (19,20,21,53,68,79,99,112) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_3292\3464066820.py:11: DtypeWarning: Columns (53,94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_3292\3464066820.py:11: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_

,BirthMonth,TimeofBirth,BirthDayofWeek,BirthPlace,MotherSingleYearofAge,MotherAge,MotherNativity,ResidenceStatus,MotherRaceRecode31,MotherRaceRecode6,...,CleftLipw/orw/oCleftPalate,CleftPalatealone,DownSyndrome,SuspectedChromosomalDisorder,Hypospadias,NoCongenitalAnomaliesChecked,InfantTransferred,InfantLivingatTimeofReport,InfantBreastfedatDischarge,SmokeStatus
0,April,2329,Tuesday,Hospital,33,30-34 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
1,May,455,Friday,Hospital,28,25-29 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
2,March,1357,Wednesday,Hospital,23,20-24 Years,Born in the U.S. (50 US States),RESIDENT,Black (only),Black (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
3,April,1506,Wednesday,Hospital,22,20-24 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
4,October,2319,Wednesday,Hospital,19,15-19 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Smoker


In [16]:
selected_columns= ['FatherAge', 'MotherAge', 'BodyMassIndex' ,'BirthWeightDetailInGrams', 'PriorBirthsNowLiving',
                   'DeliveryMethodRecode1', 'SexofInfant', 'CigarettesBeforePregnancyRecode', 'Cigarettes1stTrimesterRecode', 
                   'Cigarettes2ndTrimesterRecode', 'Cigarettes3rdTrimesterRecode',
                   'MotherHeightinTotalInches', 'BodyMassIndexRecode', 'Pre-pregnancyWeightRecode',
                   'DeliveryWeightRecode', 'WeightGainRecode','AdmissiontoNICU']

In [17]:
df = df.loc[:, [col.lower() in map(str.lower, selected_columns) for col in df.columns]]
df.shape

(999993, 17)

In [18]:
df = df.rename(columns={'AdmissiontoNICU': 'AdmissionToNICU'})

In [19]:
dtype = get_dtype_lists(df, df.columns)
dtype

{'object': ['MotherAge',
  'FatherAge',
  'PriorBirthsNowLiving',
  'CigarettesBeforePregnancyRecode',
  'Cigarettes1stTrimesterRecode',
  'Cigarettes2ndTrimesterRecode',
  'Cigarettes3rdTrimesterRecode',
  'MotherHeightinTotalInches',
  'BodyMassIndex',
  'BodyMassIndexRecode',
  'Pre-pregnancyWeightRecode',
  'DeliveryWeightRecode',
  'WeightGainRecode',
  'DeliveryMethodRecode1',
  'SexofInfant',
  'BirthWeightDetailInGrams',
  'AdmissionToNICU']}

In [20]:
#numerics = dtype["int64"]
categories = dtype["object"]

In [21]:
for category in tqdm(categories): 
    df[category] = df[category].apply(str)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  6.67it/s]


In [22]:
minimum_support = 1 / len(df)
minimum_support

1.0000070000490004e-06

***Hypothesis 1: Infants born to mothers who smoked cigarettes before pregnancy have a higher likelihood of being born preterm.***

In [23]:
from itertools import combinations, product

In [24]:
outcome_categories = ['AdmissionToNICU']
predictor_categories = ['CigarettesBeforePregnancyRecode']

In [25]:
predictor_tuples = {}
#extract the value tuple for each category
for category in tqdm(predictor_categories): 
    predictor_tuples[category] = [unit for unit in get_category_value_tuples(df, [category]) if get_support(df,[unit]) >= minimum_support]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.92s/it]


In [26]:
#if you change the lenght, you won't get the leverage and Jaccard distance
#we did the calculation considering a relationship between 1 outcome and 1 predictor
length = 1
rows = []

#get all possible outcome categories
outcomes = get_category_value_tuples(df, outcome_categories)

#loop through all possible predictor combinations of length = 1
for predictor_combination in tqdm(combinations(predictor_categories,length)):
    
    # loop through a list of relationships for the given predictor_combination and outcome
    for relationship in product(*[predictor_tuples[category] for category in list(predictor_combination)], outcomes):
        
        # separate the predictor and outcome values from the relationship list
        predictor = list(relationship[0:length])
        outcome = relationship[length]
        
        try: 
            support = get_support(df, list(relationship))
        
            print('relationship', relationship)
            print('predictor ' , predictor)
            print('outcome ' , outcome)
            print('support ' , support)
            print('test support ' , support >= minimum_support)
            
            # if the support meets the minimum_support criteria, store the row
            if support >= minimum_support: 
            
                row = {}
                
                # loop through each predictor and add it to the row dictionary
                for i in range(0,length): 
                    row["Predictor_%s" % i]  = " : ".join(predictor[i]) 
                    
                # add the outcome, support, confidence, lift, leverage, and conviction to the row dictionary
                row['Outcome'] = " : ".join(list(outcome)) 
                row['Support'] = support
                row['Confidence'] = get_confidence(df, predictor, [outcome])
                row['Lift'] = get_lift(df, predictor + [outcome])
                row['Leverage'] = get_leverage(df, predictor + [outcome])
                row['Conviction'] = get_conviction(df, predictor, [outcome])
                
                # determine whether the rule is interesting or not
                if (row['Lift'] > 1) | (row['Leverage'] > 0) | (row['Conviction'] > 1):
                    row['Interestingness'] = 1
                    if jaccard_distance(graph,str(outcome[0]),str(predictor[0][0])) < 0.2:
                        row['Semantic_Interestingness'] = 1
                    else :
                        row['Semantic_Interestingness'] = 0
                else :
                    row['Interestingness'] = 0
                    row['Semantic_Interestingness'] = 0
                
                
                # add the row to the list of rows
                rows += [row]
                print('row ',row, end='\n \n')
            
        except:
            pass

0it [00:00, ?it/s]

relationship (('CigarettesBeforePregnancyRecode', 'Nonsmoker'), ('AdmissionToNICU', 'No'))
predictor  [('CigarettesBeforePregnancyRecode', 'Nonsmoker')]
outcome  ('AdmissionToNICU', 'No')
support  0.8739531176718237
test support  True
row  {'Predictor_0': 'CigarettesBeforePregnancyRecode : Nonsmoker', 'Outcome': 'AdmissionToNICU : No', 'Support': 0.8739531176718237, 'Confidence': 0.9130826348106179, 'Lift': 1.0031732380916625, 'Leverage': 0.002764488941709664, 'Conviction': 1.033229993151831, 'Interestingness': 1, 'Semantic_Interestingness': 1}
 
relationship (('CigarettesBeforePregnancyRecode', 'Nonsmoker'), ('AdmissionToNICU', 'Yes'))
predictor  [('CigarettesBeforePregnancyRecode', 'Nonsmoker')]
outcome  ('AdmissionToNICU', 'Yes')
support  0.08119156834097839
test support  True
row  {'Predictor_0': 'CigarettesBeforePregnancyRecode : Nonsmoker', 'Outcome': 'AdmissionToNICU : Yes', 'Support': 0.08119156834097839, 'Confidence': 0.08482675974962885, 'Lift': 0.9684678947151505, 'Leverage'

1it [07:42, 462.35s/it]


In [27]:
table = pd.DataFrame(rows, columns=["Predictor_" + str(i) for i in range(0,length)] + ['Outcome','Support','Confidence','Lift','Leverage','Conviction','Interestingness','Semantic_Interestingness'])

In [30]:
#test the hypothesis
where = (table.Outcome == "AdmissionToNICU : Yes") & ((table.Predictor_0 != "CigarettesBeforePregnancyRecode : Nonsmoker")&(table.Predictor_0 != "CigarettesBeforePregnancyRecode : Unknown or not stated"))
table[where].round(5).sort_values("Confidence", ascending=False)

,Predictor_0,Outcome,Support,Confidence,Lift,Leverage,Conviction,Interestingness,Semantic_Interestingness
7,CigarettesBeforePregnancyRecode : 6-10,AdmissionToNICU : Yes,0.00205,0.15130,1.72740,0.00086,1.07507,1,1
13,CigarettesBeforePregnancyRecode : 21-40,AdmissionToNICU : Yes,0.00019,0.14571,1.66352,0.00008,1.06803,1,1
4,CigarettesBeforePregnancyRecode : 1-5,AdmissionToNICU : Yes,0.00205,0.14321,1.63500,0.00079,1.06491,1,1
10,CigarettesBeforePregnancyRecode : 11-20,AdmissionToNICU : Yes,0.00144,0.14064,1.60574,0.00054,1.06174,1,1
19,CigarettesBeforePregnancyRecode : 41 or more,AdmissionToNICU : Yes,0.00004,0.13213,1.50855,0.00001,1.05133,1,1


***Interpretation References*** <br>
* Support :
    * The number of instances that support the relationship.
    * Support > min_Support means there are more than 1 instance that support this relationship.
* Confidence :
    * After removing probability of predictors occuring alone, what is the likelihood of the outcome value appearing with the predictor value anyway?
* Lift :
    * Lift > 1 means that items are positively correlated and occur together more often than what we would expect by chance alone. It's considered as an indication of an interesting association rule.
    * Lift = 1 means there is no relationship; The category values appear independently on their own.
    * Lift < 1 means that items are negatively correlated and occur together less often than what we would expect by chance alone. It's is considered as an indication of an uninteresting association rule.
* Leverage :
    * Leverage measures the difference between the observed frequency of co-occurrence and the frequency that would be expected if the items were independent. It can range from -1 to 1.

    * Leverage = 0 means that the items are independent of each other. 
    * Leverage > 0 means that the items are positively associated. It's considered as an indication of an interesting association rule.
    * Leverage < 0 means that the items are negatively associated. It's considered as an indication of an uninteresting association rule.

* Conviction :
    * Conviction measures how much more likely the consequent (outcome) is to occur when the antecedent (predictor) is present, compared to when it's absent. It can range from 0 to infinity.

    * Conviction = 1 means that the predictor and outcome are independent.
    * Conviction > 1 means that the predictor and outcome are positively associated, and the higher the value, the stronger the association. It's considered as an indication of an interesting association rule.
    * Conviction < 1 means that the predictor and outcome are negatively associated, and the lower the value, the stronger the association. It's considered as an indication of an uninteresting association rule.
    
* Interestingness :
    * Interestingness = 1 : Interesting rule.
    * Interestingness = 0 : Uninteresting rule.
    
* Semantic_Interestingness:
    * It is a measure of the interestingness of a rule based on the semantic similarity between the predictor and outcome categories using the Ontology. It is calculated as the Jaccard distance between the predictor and outcome categories in a domain-specific knowledge graph. 
    * Semantic_Interestingness = 1 indicates that the predictor and outcome categories are semantically similar. The rule is then considered semantically interesting.
    * Semantic_Interestingness = 0 indicates that they are not semantically similar. The rule is then not considered semantically interesting.
    
* Jaccard Distance:
    * It is a measure of similarity between two sets. It is calculated as the ratio of the size of the intersection of the two sets to the size of the union of the two sets. In the context of association rule mining, Jaccard distance can be used to measure the similarity between the predictor and outcome categories. A low Jaccard distance value indicates that the predictor and outcome categories have a high degree of similarity, while a high Jaccard distance value indicates that they have a low degree of similarity (we fixed a threashold = 0.2).



***Conclusion*** <br>
* Hypothesis 1 accepted with interestingness.